In [1]:
from itertools import combinations 
import math

import numpy as np

## Karatsuba Multiplication

In elementary school we learn to multiply 2 numbers:

<img src="images/elementary_multiplication.jpg" alt="drawing" width="200"/>

Previous multiplication method/algorithm is teach in **elementary school** and it requires the mutliplication of all numbers. In spite of being easy to teach, its complexity is $\mathcal{O}(n^2)$.

However, to compute the product of large number the previous algorithm is inefficient. In this notebook we will implement another multplication algorithm, called **Karatsuba algorithm**, its compleixty is $\mathcal{O}(n^{1.585})$. For the curios reader there are even more efficient algorithms, so far the best one is Harvey-Hoeven algorithm with $\mathcal{O}(nlog(n))$, more information can be found in this [link](https://en.wikipedia.org/wiki/Computational_complexity_of_mathematical_operations#Arithmetic_functions).

To explain **Karatsuba algorithm** we will focus on a simple multiplication: 


$$
\begin{array} 
& & 5 & 6 & 7 & 8 \\
x & 1 & 2 & 3 & 4 \\
\hline
\end{array}
$$

**0th Step:**  
>  Decompose the numbers A and B, in half. 
$$ 5678 \to a = 56  \quad  b = 78.$$

$$ 1234 \to c=12  \quad  d=34.$$  

Instead of multiply as in elementary-school, we will decompose each number so that:  
$$ 5678·1234=(a·100+b)(c·100+d)=ac·100²+(ad+bc)·100 +bd $$

**1st Step:**  
> Compute $a·c=672$.

**2nd Step:**  
> Compute $b·d=2652$.  

_Please notice that you can't use to the elementary multiplication algorithm to computer neither $a·c$ nor $
b·c$, so you will need to implement this algorithm recursively, you can see how in the code below._  

**3rd Step:**  
> Compute $(a+b)·(c+d)=134·46=6164$.

Take a second a look at the previous computation, it will be important for the next step.  
$$(a+b)·(c+d) = ac+ad+bc+bd$$

**4th step:**  
> This is the most imporatant step, because we combine the last 3 previous steps.  

$$ 3^{th} - 2^{nd} - 1^{st} = (ac+ad+bc+bd)-bd-ac=ad+bd$$

_Using this trick, instead of multiplying $ad$ and then do another multiplication $bc$, we get the result by simply doing step 3. So, we avoid one extra multiplication at the cost of a few extra additions._

**FINAL STEP:**

If you have paid attention to each step, you will have noticed that in each step we are getting the terms that I have written down in the **0th step**. 

So, as you may figure out the result will be:  

$$ Result \to 1^{st}·B^2+4^{th}·B+2^{nd}$$

In this case it is obvious that $B=100$. However, for any other number B could be: $1, 10, 100, 1000, etc.$ So to figure out the value of $B$, you need to know the decimal place of the letter $a$.

**Example**  
Suppose we are interested in computing $97801·8040$. Our first step will be decomposing so $a=97$ and $b=801$. Since $97$ is in the thousands position, $B$ will be $B=1000$.

In [2]:
def digit_length(x):
    if x > 0:
        digits = int(math.log10(x))+1
    elif x==0: 
        digits=1
    else:
        raise ValueError("a negative number was found")
    return digits

def decompose(x, right_digits):
    if right_digits < 1 or right_digits >= digit_length(x):
        return ValueError(f"To decompose {x}, choose a number between 1 and {digit_length(x)-1}")
    decimal_number = 10**right_digits
    right = int(x%decimal_number)
    left = int((x-x%decimal_number)//decimal_number)
    return left, right

In [3]:
def karatsuba_product(x, y):
    # Base case
    if digit_length(x) == 1 or digit_length(y) == 1:
        return x*y

    # Decompose
    unit_length = min(digit_length(x)//2, digit_length(y)//2)
    a, b = decompose(x, unit_length)
    c, d = decompose(y, unit_length)
    
    zeros_to_add = 10**(unit_length)
    
    # STEP 1: a x c
    z1 = karatsuba_product(a, c)

    # STEP 2: b x d
    z2 = karatsuba_product(b, d)

    # STEP 3: 
    z3 = karatsuba_product(a+b, c+d)

    # STEP 4:
    z4 = z3 - z2 - z1
    
    result = z1*zeros_to_add**2 + z4*zeros_to_add + z2
    return result
    

### Exercise 1:

Verify that Karatsuba algorithm works.

In [4]:
x, y = 5678, 1234
karatsuba_product(x,y)

7006652

In [5]:
x*y

7006652

Good!  
Karatsuba and python multiplication give the same result.

### Exercise 2: 

Karatsuba algorithm is useful for large number, try to compute x·y.

Where x=3141592653589793238462643383279502884197169399375105820974944592 and y=2718281828459045235360287471352662497757247093699959574966967627.

In [6]:
a=3141592653589793238462643383279502884197169399375105820974944592
b=2718281828459045235360287471352662497757247093699959574966967627
result=karatsuba_product(a, b)

In [7]:
result

8539734222673567065463550869546574495034888535765114961879601127067743044893204848617875072216249073013374895871952806582723184

In [8]:
a*b

8539734222673567065463550869546574495034888535765114961879601127067743044893204848617875072216249073013374895871952806582723184

In [9]:
a*b==result

True

Good!
Karatsuba and python multiplication give the same result.

### Exercise 3:

Test your code, run at least 1000 samples with different length and verify all the results.

In [10]:
def python_product(x, y):
    return np.multiply(x,y)

In [18]:
# To have reproducable results
np.random.seed(3)

# Generating numbers
low, high = 10**5, 10**9
samples = np.random.randint(low, high, size=1000, dtype=np.int64)

# Verifying all numbers generated will be multiplied correctly.
for x, y in combinations(samples, 2):    
    correct = karatsuba_product(x, y) == x*y
    if not correct:
        raise ValueError(f"A bug is found for x = {x}, y= {y}")
print("Pass! All multpliplications are correct!")

Pass! All multpliplications are correct!


### Mistakes

1 . For some reason, np.log10(x) outputs the following error when I was trying to get the number length of a large number x. I guess numpy has efficient ways to do the logarithm that are not compatible with such huge number, so I decide to use the standard math library for Python.

In [13]:
# Numpy is not able to compute this number (at least my numpy version)
x=3141592653589793238462643383279502884197169399375105820974944592
np.log10([x])

TypeError: loop of ufunc does not support argument 0 of type int which has no callable log10 method

2. Be careful, in exercise 3. I try generating larger numbers, but standard python multiplication did overflow, computing a wrong result.

In [19]:
np.random.seed(3)

low, high = 10**9, 10**12
samples = np.random.randint(low, high, size=1000, dtype=np.int64)

# Verifying all numbers generated will be multiplied correctly.
for x, y in combinations(samples, 2):    
    correct = karatsuba_product(x, y) == x*y
    print(x, y)
    print(x*y)
    if not correct:
        raise ValueError(f"A bug is found for x = {x}, y = {y}")
print("Pass! All multpliplications are correct!")

456570294424 791795084744
-9175391515681902912


<ipython-input-19-ab0c3278c064>:8: RuntimeWarning: overflow encountered in long_scalars
  correct = karatsuba_product(x, y) == x*y
<ipython-input-19-ab0c3278c064>:10: RuntimeWarning: overflow encountered in long_scalars
  print(x*y)


ValueError: A bug is found for x = 456570294424, y = 791795084744

In [15]:
1846599218847425409*2385050807916877375

4404232958810726626061632997512221375

As you can see, in the loop Python give us negative number (-9175391515681902912), but it is impossible. The only reasonable explanation is that due to an overflow Python is not able to compute the product properly.